In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import date
import sklearn

In [2]:
df = pd.read_csv(r"C:\Users\drsco\Documents\GitHub\QTW\qtw\case study 6\all_train.csv.gz", compression='gzip', 
                                 header=0, sep=',', quotechar='"')


In [3]:
df.rename(columns={ df.columns[0]: "label" }, inplace = True)
df.head()

,label,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f21,f22,f23,f24,f25,f26,mass
0,1.0,-0.346368,0.416306,0.999236,0.475342,0.427493,-0.005984,1.989833,0.344530,1.566297,...,4.105282,0.267826,0.378718,1.743123,3.406367,4.350537,-0.352571,1.130032,2.227706,1000.0
1,1.0,1.708236,-0.319394,-1.241873,-0.887231,-0.871906,-0.005984,-0.001047,-1.038225,0.655748,...,-1.178141,-0.877361,-1.483769,-0.573682,-1.693781,-0.545062,-0.299118,-0.662942,-0.193019,750.0
2,0.0,-0.360693,1.794174,0.264738,-0.472273,-0.292344,-1.054221,-1.150495,1.423404,1.270098,...,-1.199511,0.539020,-1.590629,-0.573682,-0.543636,-0.937456,-0.300344,-0.523262,-1.506304,750.0
3,1.0,-0.377914,-0.103932,-0.649434,-2.125015,-1.643797,-0.005984,1.011112,-1.040340,-0.541991,...,0.463763,-0.006583,1.089122,-0.573682,-0.276348,-0.409272,-0.349926,-0.307123,0.529698,1250.0
4,0.0,-0.067436,-0.636762,-0.620166,-0.062551,1.588715,-0.005984,-0.595304,-1.238987,0.336844,...,-0.552837,-1.418494,-0.562982,1.743123,0.881802,0.002516,1.560950,-0.150760,-1.023889,750.0


In [4]:

df.isna().sum()

label    0
f0       0
f1       0
f2       0
f3       0
f4       0
f5       0
f6       0
f7       0
f8       0
f9       0
f10      0
f11      0
f12      0
f13      0
f14      0
f15      0
f16      0
f17      0
f18      0
f19      0
f20      0
f21      0
f22      0
f23      0
f24      0
f25      0
f26      0
mass     0
dtype: int64

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, holdback_index in split.split(df, df.label):
    training_df = df.iloc[train_index]
    holdback_df = df.iloc[holdback_index]

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for validation_index, test_index in split.split(holdback_df, holdback_df.label):
    validation_df = holdback_df.iloc[validation_index]
    test_df = holdback_df.iloc[test_index]

print("Size of training set: ",training_df.shape)
print("Size of validation set: ",validation_df.shape)
print("Size of test set: ",test_df.shape)

Size of training set:  (5600000, 29)
Size of validation set:  (700000, 29)
Size of test set:  (700000, 29)


In [7]:
train_features = training_df.drop(['label'],axis=1).values
train_labels = training_df['label'].values

validation_features = validation_df.drop(['label'],axis=1).values
validation_labels = validation_df['label'].values

test_features = test_df.drop(['label'],axis=1).values
test_labels = test_df['label'].values

In [8]:
from sklearn.preprocessing import StandardScaler
scl_obj = StandardScaler()
scl_obj.fit(train_features)
train_features_scaled = scl_obj.transform(train_features)
scl_obj.fit(validation_features)
validation_features_scaled = scl_obj.transform(validation_features)
scl_obj.fit(test_features)
test_features_scaled = scl_obj.transform(test_features)

# Simple Network

In [9]:
import tensorflow as tf
from tensorflow.keras import regularizers

research_model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, activation='tanh', input_shape=(28,),kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(300, activation='tanh',kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(300, activation='tanh',kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(300, activation='tanh',kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(300, activation='tanh',kernel_regularizer=regularizers.l2(0.00001)),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

research_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(name='AUC'),'accuracy'])

In [10]:
research_model.fit(train_features_scaled, train_labels, batch_size=5000, epochs=5)

Epoch 1/5


c:\Users\drsco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1120/1120 [==============================] - 404s 357ms/step - loss: 0.3016 - AUC: 0.9469 - accuracy: 0.8664
Epoch 2/5
1120/1120 [==============================] - 385s 343ms/step - loss: 0.2801 - AUC: 0.9542 - accuracy: 0.8780
Epoch 3/5
1120/1120 [==============================] - 365s 326ms/step - loss: 0.2751 - AUC: 0.9554 - accuracy: 0.8801
Epoch 4/5
1120/1120 [==============================] - 361s 322ms/step - loss: 0.2723 - AUC: 0.9561 - accuracy: 0.8810
Epoch 5/5
1120/1120 [==============================] - 348s 311ms/step - loss: 0.2703 - AUC: 0.9565 - accuracy: 0.8820


In [11]:
callback = [
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10,mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.05)
]